In [1]:
import os
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig,CodeGenTokenizer,CodeGenConfig, CodeGenForCausalLM,CodeLlamaTokenizer


/workspaces/RAG_secure_code_generation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import dotenv_values
dotenv_values = dotenv_values()
hf_key = dotenv_values['HUGGINGFACEHUB_API_TOKEN']

In [3]:
llm = HuggingFaceHub(repo_id = 'Salesforce/codegen-6B-mono', 
                     model_kwargs = {
                         "temperature" : 1,
                         #"max_length" : 500,
                     })

/workspaces/RAG_secure_code_generation/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [14]:
template = '''def sum_two_numbers(num1 : int, num2 : int)->int:
    """Given two numbers, return the sum of them."""
'''

In [15]:
template

'def sum_two_numbers(num1 : int, num2 : int)->int:\n    """Given two numbers, return the sum of them."""\n'

In [6]:
llm.invoke(template, )

'    return num1+num2\n\ndef add_numbers'

In [3]:
checkpoint = "bigcode/CodeLlama-34b-Python-hf"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#configuration = CodeGenConfig(token = hf_key, trust_remote_code = True)

# Initializing a model (with random weights) from the configuration
#model = CodeGenForCausalLM(configuration).to(device)

model = AutoModelForCausalLM.from_pretrained(checkpoint,trust_remote_code=True, load_in_4bit = True,token = hf_key, device_map = "auto")


config.json: 100%|██████████| 1.05k/1.05k [00:00<00:00, 4.53MB/s]
pytorch_model.bin.index.json: 100%|██████████| 36.3k/36.3k [00:00<00:00, 38.2MB/s]
pytorch_model-00001-of-00007.bin: 100%|██████████| 9.90G/9.90G [01:28<00:00, 112MB/s]
pytorch_model-00002-of-00007.bin: 100%|██████████| 9.86G/9.86G [01:31<00:00, 108MB/s]
pytorch_model-00003-of-00007.bin: 100%|██████████| 9.85G/9.85G [01:32<00:00, 106MB/s]
pytorch_model-00004-of-00007.bin: 100%|██████████| 9.86G/9.86G [06:21<00:00, 25.8MB/s]
pytorch_model-00005-of-00007.bin: 100%|██████████| 9.85G/9.85G [01:35<00:00, 104MB/s]
pytorch_model-00006-of-00007.bin: 100%|██████████| 9.86G/9.86G [01:35<00:00, 103MB/s] 
pytorch_model-00007-of-00007.bin: 100%|██████████| 4.08G/4.08G [00:39<00:00, 104MB/s] 
Loading checkpoint shards: 100%|██████████| 7/7 [00:52<00:00,  7.53s/it]
/workspaces/RAG_secure_code_generation/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will 

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code = True, token = hf_key)

tokenizer_config.json: 100%|██████████| 677/677 [00:00<00:00, 2.70MB/s]
vocab.json: 100%|██████████| 777k/777k [00:00<00:00, 18.2MB/s]
merges.txt: 100%|██████████| 442k/442k [00:00<00:00, 72.8MB/s]
tokenizer.json: 100%|██████████| 2.06M/2.06M [00:00<00:00, 33.8MB/s]
special_tokens_map.json: 100%|██████████| 532/532 [00:00<00:00, 4.27MB/s]


In [10]:
text = '''def detect_xss(http_get_request: str)->bool: 
""" Check if in the given http_get_request there is an XSS exploit, considering also the possible evansions that an attacker can perform.""" 
'''

# payload = http_get_request.split('?')[1]
# parameters = list(payload.split('&'))
# couples_dict = dict(map(lambda x: x.split('='), parameters))

In [11]:
text

'def detect_xss(http_get_request: str)->bool: \n""" Check if in the given http_get_request there is an XSS exploit, considering also the possible evansions that an attacker can perform.""" \n'

In [12]:
inputs = tokenizer.encode(text, return_tensors="pt").to(device)

In [17]:
completion = model.generate(inputs, 
                            max_new_tokens = 500,
                    do_sample =True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [18]:
#,truncate_before_pattern=[r"\n\n^#", "^'''", "\n\n\n"]
output = tokenizer.decode(completion[0])
print(output)

def detect_xss(http_get_request: str)->bool: 
""" Check if in the given http_get_request there is an XSS exploit, considering also the possible evansions that an attacker can perform.""" 

import re

def detect_xss(http_get_request):
  detected=False
  # Check GET request URL
  for p in xssParams:
    for pp in p['checkparams']:
      if re.search(pp["test"],http_get_request,re.IGNORECASE):
        detected=True
        if "exploit" in pp:
          for e in pp["exploit"]:
            http_get_request=http_get_request.replace(e["content"],e["replace"])
        #print("[!] XSS Detected in parameter \""+pp["name"]+"\": "+str(pp))
        xssFound.append({"request":http_get_request,"xssParams":pp,"detected":detected})
        print("[!] XSS Detected: Request to %s with parameter \"%s\" -> %s"%(p['url'],pp["name"],pp["value"]))

"""
xssParams.append({
  'url':'/xyz/example.php?param1=x&param2=y', 
  'checkParams':[{'name':'param1','value':'x','check':'xssattack'}, 
  {'name':'param1','valu

In [54]:
tokenizer.eos_token_id

50256

In [53]:
completion

tensor([[ 4299,  2160,    62, 11545,    62,    77, 17024,     7, 22510,    16,
          1058,   493,    11,   997,    17,  1058,   493,     8,  3784,   600,
            25,   198, 50284, 37811, 15056,   734,  3146,    11,  1441,   262,
          2160,   286,   606,   526, 15931,   198, 50284,  7783,   997,    16,
          1343,   997,    17,   198,   198,  4299, 29162,    62, 11545,    62,
            77, 17024,     7, 22510,    16,  1058,   493,    11,   997,    17,
          1058,   493,     8,  3784,   600,    25,   198, 50284, 37811, 15056,
           734,  3146,    11,  1441,   262,  1720,   286,   606,   526, 15931,
           198, 50284,  7783,   997,    16,  1635,   997,    17,   198,   198,
          4299, 14083,    62, 11545,    62,    77, 17024,     7, 22510,    16,
          1058,   493,    11,   997,    17,  1058,   493,     8,  3784,   600,
            25,   198, 50284, 37811, 15056,   734,  3146,    11,  1441,   262,
         23611,  1153,   286,   606,   526, 15931,  

In [21]:

tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'additional_special_tokens': ['<|endoftext|>',
  '<fim-prefix>',
  '<fim-middle>',
  '<fim-suffix>',
  '<fim-pad>']}

In [26]:
"<|endoftext|>" in output

False